In [1]:
import pathlib
import os
import shutil

import utils
import artifact
import artifact_extractor
import PIL

from typing import Union
import datetime
PathLike = Union[str, os.PathLike]

In [4]:
save_dir = pathlib.Path().cwd() / "_new_artifact_temp"
save_dir.mkdir(exist_ok=True, parents=True)

artifact_source_dir = pathlib.Path("~/Desktop/source").expanduser()

artifact_crop_output_dir = save_dir / "artifact_crop"
artifact_component_output_dir = save_dir / "artifact_components"
ocr_output_dir = save_dir / "ocr_output"
good_format_output_path = save_dir / "artifacts_good_format.json"

In [3]:
artifact_extractor.copy_tesseract_font()

In [5]:
artifact_extractor.crop_new_artifacts_multiprocess(artifact_source_dir, artifact_crop_output_dir)

  0%|          | 0/574 [00:00<?, ?it/s]

In [3]:
artifact_extractor.get_artifact_components(artifact_crop_output_dir, output_dir=artifact_component_output_dir)

Getting artifact component crops...


  0%|          | 0/574 [00:00<?, ?it/s]

In [4]:
artifacts = artifact_extractor.run_ocr_on_artifact_components_multiprocess(
    artifact_component_output_dir, ocr_output_dir
)

Running OCR...


  0%|          | 0/574 [00:00<?, ?it/s]

In [5]:
# Save artifacts to good format
artifacts = artifact_extractor.load_json(ocr_output_dir / "artifacts.json")
all_artifacts = artifact_extractor.remove_duplicate_artifacts(artifacts=artifacts)


In [6]:
artifact.artifact_list_to_good_format_json(
    all_artifacts, output_path=good_format_output_path
)

In [8]:
all_artifacts[0]

Artifact(
    artifact_type="Plume of Death",
    level="0",
    rarity="5",
    main_stat="ATK",
    value="47",
    set_name="Marechaussee Hunter",
    substats="{'ATK%': 5.3, 'Elemental Mastery': 19.0, 'DEF%': 6.6, 'HP': 239.0}",
    equipped="None",
    artifact_id="IMG_4197",
    file_path="None",
)

In [3]:
path = pathlib.Path("/Users/Christian/Desktop/escape map.png")

In [9]:
x = path.stat().st_ctime

In [10]:
datetime.datetime.fromtimestamp(x)

datetime.datetime(2023, 10, 23, 11, 28, 15, 217377)

In [12]:
y = datetime.datetime(2023, 1, 1, 0, 0, 0).timestamp()
y

1672560000.0

In [13]:
path.stat().st_ctime = y

AttributeError: readonly attribute